# Linear Regression Implementation (Single variable)

In [40]:
import tensorflow as tf
import numpy as np
import pandas as pd

# Use plotly as it is an interaction plot
import plotly.express as px
# sub plot
from plotly.subplots import make_subplots
import plotly.graph_objects as go

### Load data

In [41]:
# Load data from TF dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path='boston_housing.npz', test_split=0.2, seed=113
)

In [42]:
# We got X_train np.array with row:404, col:13
print(f"X Shape: {X_train.shape}, X Type:{type(X_train)})")
# We got y_train np.array with row:404
print(f"y Shape: {y_train.shape}, y Type:{type(y_train)})")

X Shape: (404, 13), X Type:<class 'numpy.ndarray'>)
y Shape: (404,), y Type:<class 'numpy.ndarray'>)


In [43]:
# To make sure you understand the data. Read dataspec first.

# Variables in order:

#  X_dataset
#  CRIM     per capita crime rate by town
#  ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
#  INDUS    proportion of non-retail business acres per town
#  CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
#  NOX      nitric oxides concentration (parts per 10 million)
#  RM       average number of rooms per dwelling
#  AGE      proportion of owner-occupied units built prior to 1940
#  DIS      weighted distances to five Boston employment centres
#  RAD      index of accessibility to radial highways
#  TAX      full-value property-tax rate per $10,000
#  PTRATIO  pupil-teacher ratio by town
#  B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
#  LSTAT    % lower status of the population

#  y_dataset
#  MEDV     Median value of owner-occupied homes in $1000's

In [44]:
# Construct np.array to pd.Dataframe for easier handling as it is more human readable
X_df = pd.DataFrame(X_train, columns=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT'])
y_df = pd.DataFrame(y_train, columns=['MEDV'])

### Take a look at dataset

In [45]:
X_df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,1.23247,0.0,8.14,0.0,0.5380,6.142,91.7,3.9769,4.0,307.0,21.0,396.90,18.72
1,0.02177,82.5,2.03,0.0,0.4150,7.610,15.7,6.2700,2.0,348.0,14.7,395.38,3.11
2,4.89822,0.0,18.10,0.0,0.6310,4.970,100.0,1.3325,24.0,666.0,20.2,375.52,3.26
3,0.03961,0.0,5.19,0.0,0.5150,6.037,34.5,5.9853,5.0,224.0,20.2,396.90,8.01
4,3.69311,0.0,18.10,0.0,0.7130,6.376,88.4,2.5671,24.0,666.0,20.2,391.43,14.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,0.21977,0.0,6.91,0.0,0.4480,5.602,62.0,6.0877,3.0,233.0,17.9,396.90,16.20
400,0.16211,20.0,6.96,0.0,0.4640,6.240,16.3,4.4290,3.0,223.0,18.6,396.90,6.59
401,0.03466,35.0,6.06,0.0,0.4379,6.031,23.3,6.6407,1.0,304.0,16.9,362.25,7.83
402,2.14918,0.0,19.58,0.0,0.8710,5.709,98.5,1.6232,5.0,403.0,14.7,261.95,15.79


In [46]:
y_df

,MEDV
0,15.2
1,42.3
2,50.0
3,21.1
4,17.7
...,...
399,19.4
400,25.2
401,19.4
402,19.4


In [47]:
# Plot and see relationship

def plot_relation(X: pd.DataFrame, y: pd.DataFrame, columns):
    '''
    Plot relation between X input and y target
    
    Args:
        X (pd.DataFrame (m,n))  : Data, m examples, n features
        y (pd.DataFrame (m,1))  : target values, m values
        columns (int)             : number of desired subplot column
        
    Output
        Interation graph
    
    '''
    
    m = X.shape[1]
    rows = m // columns     # Get row
    frac = m % columns      # Get fractual
    row = 0
    col = 1

    if frac > 0:
        rows += 1
            
    fig = make_subplots(rows=rows, cols=columns)

    for i in range(m):
            
        if row >= rows:
            row = 1
            col += 1
        else:
            row += 1
        
        fig.add_trace(go.Scatter(
            x=X.iloc[:,i],
            y=y[y.columns[0]],
            mode='markers',
            name=X.columns[i],
            customdata=X.index.values,                                  # Add customdata for data's row index for more convinient to analysis
            hovertemplate="index:%{customdata} (X: %{x}, y: %{y})"
        ), row=row, col=col)

    fig.update_layout(height=400 * rows, width=600 * columns, title_text='Relationship between All features / ' + y.columns.values[0])
    fig.show()
    
plot_relation(X_df, y_df, 2)

You will get some ideas for example.
Prices tend to be higer as crime rate(CRIM) lower

# Linear regression

##### Formula

$ f_{w,b}(x^{(i)}) = wx^{(i)} + b $

In [48]:
# Slice to smaller dataset for simplify explanation
X_train_s = X_train[[395, 332, 317], 5]
y_train_s = y_train[[395, 332, 317]]

In [49]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    go.Scatter(
        x=X_train_s,
        y=y_train_s,
        mode='markers',
        name=X_df.columns[5]
    ))

# include shapes in layout
fig.update_layout(height=400, width=600, title_text="Target")
fig.show()

In [50]:
# Loop implementation
def compute_linear_regression(X, w, b):
    """
    Computes the prediction of a linear model
    Input X compute with parameter w,b
    return prediction value
    
    Args:
        X (ndarray (m,n)): Data, m examples 
        w,b (scalar)    : model parameters  
    Returns
        f_wb (ndarray (m,)): predict values
    """
    
    m = X.shape[0]
    f_wb = np.zeros(m)
    
    for i in range(m):
        f_wb[i] = w * X[i] + b
        
    return f_wb

In [51]:
w_init = 10
b_init = -40
f_wb = compute_linear_regression(X=X_train_s, w=w_init, b=b_init)
f_wb

array([ 5.19, 25.52, 47.25])

In [52]:
# Vectorized implementation
def compute_linear_regression_v(X, w, b):
    f_wb = w * X + b
    return f_wb

In [53]:
f_wb = compute_linear_regression_v(X=X_train_s, w=w_init , b=b_init)
f_wb

array([ 5.19, 25.52, 47.25])

In [54]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    go.Scatter(
        x=X_train_s,
        y=y_train_s,
        mode='markers',
        name=X_df.columns[5]
    ))

fig.add_trace(
    go.Line(
        x=X_train_s,
        y=f_wb,
        name='Predict'
    ))

shapes = []

m = X_train_s.shape[0]

for i in range(m):
    shapes.append(
        go.layout.Shape(
            type="line",
            x0=X_train_s[i],
            y0=f_wb[i],
            x1=X_train_s[i],
            y1=y_train_s[i],
            line=dict(
                #color=np.random.choice(colors,1)[0],
                color = 'black',
                width=1),
            opacity=1,
            layer='above'
            )
        )

# include shapes in layout
fig.update_layout(shapes=shapes)
fig.update_layout(height=400, width=600, title_text="Linear regression")
fig.show()

/Users/krit/homebrew/Caskroom/miniforge/base/envs/mlp-env/lib/python3.10/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




# Cost function

$ J(w, b) = \frac{1}{2m} \sum_{i=1}^{m} (f_{w,b}(x^{(i)}) - y^{(i)})^2 $

In [55]:
def compute_cost(X, y, w, b):
    """
    Computes the prediction of a linear model
    Args:
        X (ndarray (m,n)) : Data, m examples
        y (ndarray (m,))  : target values
        w,b (scalar)      : model parameters  
    Returns
        cost (int)        : cost error value
    """
    
    m = X.shape[0]    

    f_wb = compute_linear_regression(X, w, b)
    
    error_sum = 0
    
    for i in range(m):
        error_sum += (f_wb[i] - y[i]) ** 2
        cost = error_sum / (2 * m)
    return cost

In [56]:
compute_cost(X_train_s, y_train_s, w=w_init, b=b_init)

7.766500000000009

In [57]:
def compute_cost_v(X, y, w, b):
    f_wb = compute_linear_regression_v(X, w, b)
    cost = ((f_wb - y) ** 2).mean() / 2
    return cost

In [58]:
compute_cost_v(X_train_s, y_train_s, w=w_init, b=b_init)

7.766500000000009

Gradeint Descent

$ \{ $
    
$ w^{(i)}_j := w^{(i)}_j - \alpha \frac{\sigma}{\sigma w}J(w, b)x^{(i)}_j $

$ b^{(i)} := b^{(i)} - \alpha \frac{\sigma}{\sigma w}J(w, b) $

$ \} {stimulous update} $


$ \frac{\sigma}{\sigma w}J(w, b) = \frac{1}{m} \sum_{i=1}^{m} (h_{\theta}(x^{(i)}) - y^{(i)}) $

In [59]:
def gradient_function(X, y, w, b):
    """
    Computes the prediction of a linear model
    Args:
        X (np.array (m,n))      : Data, m examples
        y (np.array (m,))       : target values
        w,b (scalar)            : init model parameters  
    Returns
        dj_dw, dj_db (scalar)   : tuned model parameters  
    """
    
    m = X.shape[0]
    dj_dw = 0
    dj_db = 0

    for i in range(m):
        f_wb = compute_linear_regression(X, w, b)
        dj_dw += (f_wb[i] - y[i]) * X[i]
        dj_db += f_wb[i] - y[i]
    dj_dw = dj_dw / m
    dj_db = dj_db / m 
        
    return dj_dw, dj_db

In [60]:
a,b = gradient_function(X_train_s, y_train_s, w=10.5, b=-40)
print(f'{a} {b}')

-0.5387516666666912 -0.21400000000000338


In [61]:
def gradient_function_v(X, y, w, b):
    
    m = X.shape[0]
    
    dj_dw = 0
    dj_db = 0

    f_wb = compute_linear_regression_v(X, w, b)
    error = f_wb - y
    dj_dw = error.T.dot(X)
    dj_db = sum(error)
    dj_dw = dj_dw / m
    dj_db = dj_db / m
        
    return dj_dw, dj_db

In [62]:
a,b = gradient_function_v(X_train_s, y_train_s, w=10.5, b=-40)
print(f'{a} {b}')

-0.538751666666691 -0.21400000000000338


In [63]:
def gradient_descent(X, y, w, b, alpha, num_iters, cost_function, gradient_function):
    
    j_hist = []
    p_hist = []

    for i in range(num_iters):
        
        dj_dw, dj_db = gradient_function(X, y, w, b)
        
        w = w - alpha * dj_dw
        b = b - alpha * dj_db
    
        j_hist.append(cost_function(X, y, w, b))
        p_hist.append([w,b])
        
    return w, b, j_hist, p_hist

In [64]:
w, b, j_hist, p_hist = gradient_descent(X_train_s, y_train_s, w=0, b=0 ,alpha=0.03, num_iters=10000, cost_function=compute_cost, gradient_function=gradient_function)

In [65]:
j_hist

[145.25439339281104,
 65.53683381143912,
 51.13760598323708,
 48.420248269017236,
 47.792572006700475,
 47.53913647735836,
 47.353155350759984,
 47.17976190214617,
 47.009141973649065,
 46.83953848293388,
 46.67063525856048,
 46.50237382457731,
 46.334741797738964,
 46.1677350450619,
 46.00135091616627,
 45.83558703294133,
 45.67044107306615,
 45.50591073126864,
 45.34199371237099,
 45.17868773001951,
 45.01599050643139,
 44.85389977232333,
 44.692413266872855,
 44.53152873768545,
 44.37124394076295,
 44.21155664047217,
 44.05246460951372,
 43.89396562889107,
 43.73605748787934,
 43.578737983994706,
 43.42200492296353,
 43.26585611869188,
 43.110289393234844,
 42.95530257676642,
 42.800893507549,
 42.6470600319034,
 42.49380000417865,
 42.34111128672226,
 42.188991749850295,
 42.03743927181759,
 41.88645173878839,
 41.73602704480658,
 41.586163091766586,
 41.43685778938393,
 41.28810905516608,
 41.13991481438351,
 40.99227300004068,
 40.84518155284722,
 40.698638421189244,
 40.55264156

In [66]:
print(f'result cost:{min(j_hist)} with parameter w:{w} b:{b}')

result cost:1.5334350908696672 with parameter w:10.203894002783514 b:-37.832095207770344


In [67]:
compute_cost(X_train_s, y_train_s, w=w, b=b)

1.5334350908696723

#### Let's plot Cost function

In [68]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    go.Scatter(
        x=[i for i in range(len(j_hist))],
        y=j_hist,
        mode='lines+markers',
        name=X_df.columns[5]
    ))

# include shapes in layout
fig.update_layout(height=400, width=600, title_text="Cost Function")
fig.show()

In [69]:
print(f'result cost:{min(j_hist)} with parameter w:{w} b:{b}')

result cost:1.5334350908696672 with parameter w:10.203894002783514 b:-37.832095207770344


Result: Best fit value is w: 11.37946848075005 b: -39.84189472999998

Not bad


Let's check our result with parameter w,b

and plot to see how the model fit the targets

In [70]:
compute_cost(X_train_s, y_train_s, w=w, b=b)

1.5334350908696723

In [71]:
f_wb = compute_linear_regression_v(X=X_train_s, w=w , b=b)

In [72]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    go.Scatter(
        x=X_train_s,
        y=y_train_s,
        mode='markers',
        name=X_df.columns[5]
    ))

fig.add_trace(
    go.Line(
        x=X_train_s,
        y=f_wb,
        name='Predict'
    ))

shapes = []

m = X_train_s.shape[0]

for i in range(m):
    shapes.append(
        go.layout.Shape(
            type="line",
            x0=X_train_s[i],
            y0=f_wb[i],
            x1=X_train_s[i],
            y1=y_train_s[i],
            line=dict(
                #color=np.random.choice(colors,1)[0],
                color = 'black',
                width=1),
            opacity=1,
            layer='above'
            )
        )

# include shapes in layout
fig.update_layout(shapes=shapes)
fig.update_layout(height=400, width=600, title_text="Linear regression")
fig.show()